In [4]:
import numpy as np
from garuda.ops import obb_to_aa
import os

In [5]:
# !pip install garuda


obb_labels="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/dhaka_grid_train/labels"
aa_labels="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/dhaka_grid_train/labels_init_format"
os.makedirs(aa_labels, exist_ok=True)

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from garuda.ops import obb_to_aa  # Import from Garuda

# Paths
obb_labels = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/dhaka_grid_train/labels"
aa_labels = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/dhaka_grid_train/labels_aa"
os.makedirs(aa_labels, exist_ok=True)

# Process all labels
for filename in tqdm(os.listdir(obb_labels)):
    if filename.endswith(".txt"):
        obb_label_path = os.path.join(obb_labels, filename)
        aa_label_path = os.path.join(aa_labels, filename)

        # Read OBB labels
        obb_data = np.loadtxt(obb_label_path, ndmin=2)
        if obb_data.size == 0:
            continue  # Skip empty files

        # Convert OBB to AA using Garuda's function
        aa_data = obb_to_aa(obb_data)

        # deNormalize by 640×640 image size
        aa_data[:, 1:] /= 640

        # Save to aa_labels directory
        np.savetxt(aa_label_path, aa_data, fmt="%.6f")

print("✅ OBB to AA Conversion Completed Successfully!")


100%|██████████| 135/135 [00:00<00:00, 1434.36it/s]

✅ OBB to AA Conversion Completed Successfully!


In [3]:
import os
import numpy as np
from tqdm import tqdm
from garuda.ops import obb_to_aa
import cv2

# Paths
image_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/delhi_airshed/images"
obb_labels = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/delhi_airshed/labels"
formatted_labels = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/delhi_airshed/formatted_labels"

os.makedirs(formatted_labels, exist_ok=True)

# Process all label files
for index, filename in enumerate(tqdm(os.listdir(obb_labels))):
    if filename.endswith(".txt"):
        obb_label_path = os.path.join(obb_labels, filename)
        formatted_label_path = os.path.join(formatted_labels, filename)

        # Corresponding image path
        image_path = os.path.join(image_dir, filename.replace(".txt", ".tif"))

        # Check if image exists
        if not os.path.exists(image_path):
            print(f"❌ Warning: Image {image_path} not found! Skipping...")
            continue

        # Read image size (assuming grayscale/single channel)
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        if img is None:
            print(f"❌ Error loading image: {image_path}")
            continue

        height, width = img.shape[:2]
        channels = 1 if len(img.shape) == 2 else img.shape[2]

        # Read OBB labels
        obb_data = np.loadtxt(obb_label_path, ndmin=2)
        if obb_data.size == 0:
            bboxes = []
        else:
            # Convert OBB to AA (Axis-Aligned Bounding Box)
            aa_data = obb_to_aa(obb_data)

            # Convert YOLO format to Pascal VOC format
            bboxes = []
            for row in aa_data:
                label_id, x_center, y_center, bbox_width, bbox_height = row
                
                # Convert to absolute coordinates
                x_min = (x_center - (bbox_width / 2)) * width
                y_min = (y_center - (bbox_height / 2)) * height
                x_max = (x_center + (bbox_width / 2)) * width
                y_max = (y_center + (bbox_height / 2)) * height

                bboxes.append(f"{int(label_id)} {x_min:.6f} {y_min:.6f} {x_max:.6f} {y_max:.6f}")

        # Write formatted Pascal VOC annotations
        with open(formatted_label_path, "w") as f:
            f.write(f"# {index}\n")
            f.write(f"{image_path}\n")
            f.write(f"{channels} {height} {width} 1\n")
            f.write(f"0\n")
            f.write(f"{len(bboxes)}\n")
            f.write("\n".join(bboxes) + "\n")

print("✅ Data formatting completed successfully!")


100%|██████████| 196/196 [00:00<00:00, 743.44it/s]

✅ Data formatting completed successfully!
